In [1]:
import numpy as np
from numba import njit, prange

@njit(parallel=True)
def morphological_operation_numba(padded_image, kernel_size=3, erosion=True):
    """
    Morphological erosion/dilation function expects already padded input.
    """
    pad = kernel_size // 2
    rows = padded_image.shape[0] - 2 * pad
    cols = padded_image.shape[1] - 2 * pad

    output = np.zeros((rows, cols), dtype=padded_image.dtype)

    for i in prange(rows):
        for j in range(cols):
            window = padded_image[i:i+kernel_size, j:j+kernel_size]
            if erosion:
                min_val = 255  # assuming 8-bit image
                for k in range(kernel_size):
                    for l in range(kernel_size):
                        val = window[k, l]
                        if val < min_val:
                            min_val = val
                output[i, j] = min_val
            else:
                max_val = 0
                for k in range(kernel_size):
                    for l in range(kernel_size):
                        val = window[k, l]
                        if val > max_val:
                            max_val = val
                output[i, j] = max_val
    return output

def morphological_operation(image, kernel_size=3, operation='erosion'):
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size must be an odd number.")
    
    pad = kernel_size // 2
    padded_image = np.pad(image, pad, mode='constant', constant_values=0)
    
    if operation == 'erosion':
        return morphological_operation_numba(padded_image, kernel_size, erosion=True)
    elif operation == 'dilation':
        return morphological_operation_numba(padded_image, kernel_size, erosion=False)
    else:
        raise ValueError("Operation must be 'erosion' or 'dilation'.")

def test_morphological_operation():
    img = np.array([
        [10, 20, 30, 40],
        [50, 60, 70, 80],
        [90, 100, 110, 120],
        [130, 140, 150, 160]
    ], dtype=np.uint8)

    print("Original Image:")
    print(img)

    erosion_result = morphological_operation(img, kernel_size=3, operation='erosion')
    print("\nErosion Result:")
    print(erosion_result)

    dilation_result = morphological_operation(img, kernel_size=3, operation='dilation')
    print("\nDilation Result:")
    print(dilation_result)

if __name__ == "__main__":
    test_morphological_operation()


Original Image:
[[ 10  20  30  40]
 [ 50  60  70  80]
 [ 90 100 110 120]
 [130 140 150 160]]

Erosion Result:
[[ 0  0  0  0]
 [ 0 10 20  0]
 [ 0 50 60  0]
 [ 0  0  0  0]]

Dilation Result:
[[ 60  70  80  80]
 [100 110 120 120]
 [140 150 160 160]
 [140 150 160 160]]
